# About:

This code does not contain data visualizations, the reasoning behind the imputation methods used, sampling technique, feature enginering, metrics choice and the choice of model. This document creates a pipeline of operations. The arguments behind the each of the operations mentioned above are detailed in the document named documentation.pdf. 

In this document there are four functions i.e., treating missing values, label encoding, feature transformation and model building. Then the pipeline is created based on these functions. Finally, the predictions are made on the test data and are stored in the excel file.

Import Required Libraries

In [16]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.svm import SVC
from imblearn.combine import SMOTEENN
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import joblib



Create a function to treat missing values

In [17]:


def treat_missing_values(data):
    # Keeping only those columns that are required
    columns_to_keep = ['custAge', 'profession', 'marital', 'schooling', 'default', 'housing',
                       'loan', 'contact', 'month', 'day_of_week', 'campaign', 'pdays', 'previous',
                       'poutcome', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx',
                       'euribor3m', 'nr.employed', 'pmonths', 'pastEmail', 'responded']
    
    data = data[columns_to_keep]

    # Feature engineering for schooling
    schooling_category = {
        'basic.4y' : 'basic',
        'basic.6y' : 'basic',
        'basic.9y' : 'basic',
        'high.school': 'high.school',
        'illiterate':'illiterate',
        'professional.course': 'professional.course',
        'university.degree':'university.degree',
        'unknown':'unknown',
    }

    data.loc[:,'schooling'] = data['schooling'].replace(schooling_category)

    # Imputation of missing values in education based on profession
    imputation_mapping = {
        'blue-collar' : 'basic',
        'self-employed': 'illiterate',
        'technician'   : 'professional.course',
        'admin.'        : 'university.degree',
        'services'      : 'high.school',
        'management'    : 'university.degree',
        'retired'       : 'unknown',
        'entrepreneur'  : 'university.degree'
    }

    data.loc[:,'schooling'] = data['schooling'].combine_first(data['profession'].map(imputation_mapping))

    # Profession & treating missing values of age
    data.loc[:, 'employment_status'] = data['profession'].apply(lambda x: 'retired' if x == 'retired' else ('student' if x == 'student' else 'working'))

    # Imputing age values
    mean_age_retired = data.loc[data['employment_status'] == 'retired', 'custAge'].mean()
    mean_age_student = data.loc[data['employment_status'] == 'student', 'custAge'].mean()
    median_age_working = data.loc[data['employment_status'] == 'working', 'custAge'].median()

    data.loc[:,'custAge'] = np.where((data['employment_status'] == 'retired') & data['custAge'].isna(), mean_age_retired, data['custAge'])
    data.loc[:,'custAge'] = np.where((data['employment_status'] == 'student') & data['custAge'].isna(), mean_age_student, data['custAge'])
    data.loc[:,'custAge'] = np.where((data['employment_status'] == 'working') & data['custAge'].isna(), median_age_working, data['custAge'])

    # Impute random day for missing 'day_of_week' values
    data.loc[:,'day_of_week'] = data['day_of_week'].apply(lambda day: np.random.choice(['mon', 'tue', 'wed', 'thu', 'fri']) if pd.isna(day) else day)

    # Drop remaining missing values
    data = data.dropna()

    return data





Create a function to label encode the data

In [18]:
def label_encoding(data):
    # Label encoding for 'profession'
    data.loc[:,'profession'] = data['profession'].map({'student': 'Dependents', 'retired': 'Dependents', 'unemployed': 'Unemployed&Unknown', 'unknown': 'Unemployed&Unknown',
                                                 'admin.': 'Working', 'blue-collar': 'Working', 'entrepreneur': 'Working', 'housemaid': 'Working',
                                                 'management': 'Working', 'self-employed': 'Working', 'services': 'Working', 'technician': 'Working'})

    # Label encoding for 'marital'
    data.loc[:,'marital'] = data['marital'].map({'single': 'Single&Divorced', 'divorced': 'Single&Divorced', 'married': 'married', 'unknown': 'Unknown'})

    # Label encoding for 'schooling'
    data.loc[:,'schooling'] = data['schooling'].map({'basic': 'Uneducated&BasicEducation', 'high.school': 'Uneducated&BasicEducation',
                                               'illiterate': 'Uneducated&BasicEducation', 'unknown': 'Unknown',
                                               'professional.course': 'Educated', 'university.degree': 'Educated'})

    # Label encoding for 'default'
    data.loc[:,'default'] = data['default'].map({'no': 'No', 'unknown': 'Yes&Unknown', 'yes': 'Yes&Unknown'})

    # Create a copy of the DataFrame to avoid modifying the original data
    data_copy_c = data.copy()

    # Define a mapping for specific months
    quarter_mapping = {'dec': 'QuarterEnd', 'sep': 'QuarterEnd', 'jun': 'QuarterEnd', 'mar': 'QuarterEnd'}

    # Replace specified months with 'QuarterEnd' in the copied DataFrame
    data_copy_c['month_mapped'] = data_copy_c['month'].replace(quarter_mapping)

    # Replace other months with 'others' in the copied DataFrame
    data_copy_c['month_mapped'].replace(to_replace=data_copy_c['month_mapped'][~data_copy_c['month_mapped'].isin(['QuarterEnd'])].unique(), value='others', inplace=True)

    data.loc[:,'month'] = data_copy_c['month_mapped']

    # Label encoding for 'day_of_week'
    data.loc[:,'day_of_week'] = data['day_of_week'].map({'mon': 'WeekBeginning', 'tue': 'WeekBeginning', 'wed': 'WeekBeginning',
                                                   'thu': 'WeekEnding', 'fri': 'WeekEnding'})

    # Feature engineering of other variables
    # pdays
    conditions = [
        (data['pdays'] == 999),
        (data['pdays'] < 5),
        ((data['pdays'] >= 5) & (data['pdays'] <= 10)),
        (data['pdays'] > 10)
    ]

    choices = ['first visit', 'less than 5 days', '5 to 10 days', 'greater than 10 days']

    # Create the 'pduration' column based on conditions
    data.loc[:,'pduration'] = np.select(conditions, choices, default='unknown')

    # pmonths
    conditions = [
        (data['pmonths'] == 999),
        (data['pmonths'] <= 0.2),
        (data['pmonths'] > 0.2)
    ]

    choices = ['first visit', 'less than 2 months', 'greater than 2 months']

    # Create the 'pduration' column based on conditions
    data['pduration_m'] = np.select(conditions, choices, default='unknown')

    return data





Create a function for feature transformation

In [19]:
from sklearn.preprocessing import StandardScaler
import pandas as pd

def feature_transformation(data):
    # Drop target and unnecessary columns
    X = data.drop(['responded', 'pdays', 'pmonths', 'employment_status'], axis=1)
    y = data['responded']

    # One-hot encode categorical columns
    X_encoded = pd.get_dummies(X, columns=['loan', 'marital', 'schooling', 'default', 'housing', 'day_of_week',
                                           'poutcome', 'pduration', 'pduration_m', 'profession', 'month', 'contact'], drop_first=True)

    # Identify continuous columns for normalization
    continuous_columns = ['custAge', 'campaign', 'previous', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx',
                           'euribor3m', 'nr.employed', 'pastEmail']

    # Extract the continuous columns from X_encoded
    X_continuous = X_encoded[continuous_columns]

    # Instantiate StandardScaler
    scaler = StandardScaler()

    # Fit and transform the scaler on the continuous data
    X_continuous_normalized = scaler.fit_transform(X_continuous)

    # Replace the original continuous columns in X_encoded with the normalized ones
    X_encoded[continuous_columns] = X_continuous_normalized

    return X_encoded, y




Create a function for training the model

In [20]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.svm import SVC
from imblearn.combine import SMOTEENN
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

def train_propensify_model(X_encoded, y):
    # Set a random seed for reproducibility
    np.random.seed(42)

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

    # Apply SMOTEENN to the training data
    smoteenn = SMOTEENN(random_state=42)
    X_train_resampled, y_train_resampled = smoteenn.fit_resample(X_train, y_train)

    # Create a Random Forest classifier
    rf_classifier = RandomForestClassifier(random_state=42)

    # Create a Support Vector Machine (SVM) classifier with tuned parameters
    svm_classifier = SVC(kernel='rbf', C=1.0, gamma='scale', probability=True, random_state=42)

    # Ensemble the classifiers using a VotingClassifier
    ensemble_classifier = VotingClassifier(estimators=[
        ('rf', rf_classifier),
        ('svm', svm_classifier)
    ], voting='hard')  # 'hard' for probability voting

    # Fit the ensemble model on the resampled training data
    ensemble_classifier.fit(X_train_resampled, y_train_resampled)

    # Save the trained ensemble_classifier if needed
    # joblib.dump(ensemble_classifier, 'ensemble_classifier.joblib')

    return ensemble_classifier

# Example usage:
# trained_model = train_propensify_model(X_encoded, y)
# joblib.dump(trained_model, 'propensify_model.joblib')


Load training and testing dataset. Add a column (target variable) to test data to allow the above functions to work

In [21]:
train_data = pd.read_excel(r"C:\Users\HARI CHARAN\Downloads\UPGRAD_ALL\Upgrad\train.xlsx")
test_data = pd.read_excel(r"C:\Users\HARI CHARAN\Downloads\UPGRAD_ALL\Upgrad\test.xlsx")
test_data['responded'] = 10

Define the pipeline

In [22]:


 
preprocessing_pipeline = Pipeline([
    ('missing_values', FunctionTransformer(func=treat_missing_values)),
    ('label_encoding', FunctionTransformer(func=label_encoding)),
    ('feature_transformation', FunctionTransformer(func=feature_transformation)),
])

# Fit the pipeline on the training data
X_train_transformed, y_train = preprocessing_pipeline.fit_transform(train_data)

# Train model using the transformed data
trained_model = train_propensify_model(X_train_transformed, y_train)

# Save the preprocessing pipeline
joblib.dump(preprocessing_pipeline, 'preprocessing_pipeline.joblib')

# Save the trained model
joblib.dump(trained_model, 'propensify_model.joblib')


C:\Users\HARI CHARAN\AppData\Local\Temp\ipykernel_10184\1573980397.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, 'employment_status'] = data['profession'].apply(lambda x: 'retired' if x == 'retired' else ('student' if x == 'student' else 'working'))


['propensify_model.joblib']

Load the trained model and preprocessing pipeline

In [23]:

loaded_model = joblib.load('propensify_model.joblib')

preprocessing_pipeline = joblib.load('preprocessing_pipeline.joblib')

Perform the pipeline tramsformations on the test data

In [24]:
X_test_transformed, _ = preprocessing_pipeline.transform(test_data)

Make predictions

In [25]:
predictions = loaded_model.predict(X_test_transformed)

In [26]:
# Add a new column 'Predictions' to the preprocessed test data
X_test_transformed['Predicted_Response'] = predictions

# Print the DataFrame with all columns and predictions
print(X_test_transformed)

        custAge  campaign  previous  emp.var.rate  cons.price.idx  \
0     -0.188221 -0.207980  1.713222     -0.763555        1.076265   
1     -0.507809 -0.207980  1.713222     -2.231472       -2.077163   
2      1.090129 -0.565991  1.713222     -1.210312       -1.186580   
3     -1.040455 -0.565991 -0.347067      0.832007       -0.231888   
4     -0.081692 -0.565991 -0.347067     -0.125330       -0.654655   
...         ...       ...       ...           ...             ...   
32945 -0.188221 -0.565991 -0.347067      0.640540        0.721071   
32946 -0.827396  0.508043 -0.347067     -1.210312       -1.186580   
32947 -0.827396 -0.207980 -0.347067     -1.210312       -1.186580   
32948 -0.827396 -0.565991 -0.347067      0.832007        1.537150   
32949 -1.573101 -0.207980 -0.347067     -1.912359       -1.065294   

       cons.conf.idx  euribor3m  nr.employed  pastEmail  loan_unknown  ...  \
0           0.650961  -1.593395    -2.849255   1.354201         False  ...   
1           2.3

In [27]:
X_test_transformed['Predicted_Response'].value_counts()

Predicted_Response
no     23593
yes     8552
Name: count, dtype: int64

Saving the resultant data to excel

In [28]:
excel_file_path = 'test_with.predictions.xlsx'
X_test_transformed.to_excel(excel_file_path, index=False)